In [ ]:
#@title 1. 掛載 Google Drive { display-mode: "form" }\n\nfrom google.colab import drive\nimport os\n\n#@markdown ### **設定 Google Drive 專案目錄**\nDRIVE_PROJECT_ROOT = "MyQuantProject" #@param {type:"string"}\n\ntry:\n    print("正在嘗試掛載 Google Drive...")\n    drive.mount('/content/drive', force_remount=True)\n    gdrive_path = os.path.join('/content/drive/MyDrive', DRIVE_PROJECT_ROOT)\n    os.environ['GDRIVE_PROJECT_PATH'] = gdrive_path\n    os.environ['PROJECT_ROOT_PATH'] = gdrive_path\n    print(f"✅ Google Drive 掛載成功！專案雲端路徑設定為：{gdrive_path}")\n    for folder in ['data', 'models', 'results']:\n        folder_path = os.path.join(gdrive_path, folder)\n        if not os.path.exists(folder_path):\n            os.makedirs(folder_path)\n            print(f"📁 已在雲端建立資料夾：{folder_path}")\nexcept Exception as e:\n    os.environ['GDRIVE_PROJECT_PATH'] = ""\n    print(f"⚠️ 掛載 Google Drive 失敗或未授權。錯誤：{e}")

In [ ]:
#@title 2. 同步 GitHub 專案程式碼 { display-mode: "form" }\n\nimport os\n\n#@markdown ### **1. 設定 GitHub 儲存庫**\nGITHUB_REPO_URL = "https://github.com/hsp1234-web/qlib.git" #@param {type:"string"}\n\n#@markdown ### **2. 輸入分支名稱**\nGIT_BRANCH = "main" #@param {type:"string"}\n\nos.environ['GITHUB_REPO_URL'] = GITHUB_REPO_URL\nos.environ['GIT_BRANCH'] = GIT_BRANCH\n\nrepo_name = GITHUB_REPO_URL.split('/')[-1].replace('.git', '')\nlocal_repo_path = os.path.join('/content', repo_name)\n\nif os.path.exists(local_repo_path):\n    print(f"專案資料夾已存在，正在切換至 '{GIT_BRANCH}' 分支並更新...")\n    %cd {local_repo_path}\n    !git fetch -q origin\n    !git checkout -q {GIT_BRANCH}\n    !git pull -q origin {GIT_BRANCH}\n    %cd /content\nelse:\n    print(f"專案資料夾不存在，正在從 '{GIT_BRANCH}' 分支下載整個專案...")\n    !git clone -q --branch {GIT_BRANCH} {GITHUB_REPO_URL}\n\nprint(f"\n✅ 程式碼同步完成！")

In [ ]:
#@title 3. 智慧安裝 Python 套件 (使用 uv) { display-mode: "form" }\n\nimport os\nimport sys\nimport pkg_resources\nimport importlib\n\nprint("📦 正在檢查並安裝 uv...")\n!pip install -q --no-warn-script-location uv\nprint("✅ uv 已準備就緒。")\n\nGITHUB_REPO_URL = os.environ.get('GITHUB_REPO_URL')\nif not GITHUB_REPO_URL:\n    raise ValueError("錯誤：找不到 GITHUB_REPO_URL 環境變數。請先執行儲存格 2。")\n\nrepo_name = GITHUB_REPO_URL.split('/')[-1].replace('.git', '')\nrequirements_path = os.path.join('/content', repo_name, 'requirements.txt')\n\nif not os.path.exists(requirements_path):\n    print(f"⚠️ 警告：在 '{requirements_path}' 中找不到 requirements.txt 檔案。")\nelse:\n    print("\n🔍 正在分析所需套件...")\n    importlib.reload(pkg_resources)\n    installed_packages = {pkg.key: pkg.version for pkg in pkg_resources.working_set}\n    to_install = []\n    with open(requirements_path, 'r') as f:\n        for line in f:\n            line = line.strip()\n            if not line or line.startswith('#'): continue\n            try:\n                req = pkg_resources.Requirement.parse(line)\n                if req.key not in installed_packages or not req.specifier.contains(installed_packages.get(req.key, '0.0.0')):\n                    to_install.append(line)\n            except pkg_resources.RequirementParseError:\n                to_install.append(line)\n    if not to_install:\n        print("\n✅ 所有 Python 套件皆已滿足。")\n    else:\n        print(f"\n🚀 偵測到 {len(to_install)} 個需要安裝/更新的套件，開始安裝...")\n        packages_str = " ".join(f'\"{pkg}\"' for pkg in to_install)\n        !uv pip install --quiet {packages_str}\n        print("\n✅ 環境安裝完成！")

In [ ]:
#@title 4. 執行回測 (指令列模式) { display-mode: "form" }\n\nimport os\n\n#@markdown ### **回測參數設定**\nTICKER = "^TWII" #@param {type:"string"}\nSTART_DATE = "2020-01-01" #@param {type:"date"}\nEND_DATE = "2023-01-01" #@param {type:"date"}\nSTRATEGY = "defaultlong" #@param ["defaultlong", "defaultshort", "defaultall"]\n\nGITHUB_REPO_URL = os.environ.get('GITHUB_REPO_URL')\nif not GITHUB_REPO_URL:\n    raise ValueError("錯誤：找不到 GITHUB_REPO_URL。")\n\nrepo_name = GITHUB_REPO_URL.split('/')[-1].replace('.git', '')\nscript_path = f"/content/{repo_name}/src/backtest_runner.py"\n\nif not os.path.exists(script_path):\n    raise FileNotFoundError(f"錯誤：找不到回測腳本 '{script_path}'。")\n\ncommand = f"python {script_path} --ticker \"{TICKER}\" --start_date \"{START_DATE}\" --end_date \"{END_DATE}\" --strategy {STRATEGY}"\nprint(f"執行命令: {command}")\n!{command}\nprint("\n✅ 回測執行完畢！")

In [ ]:
#@title 4.1 (新功能) 啟動互動式回測設定儀表板 { display-mode: "form" }\n\n# ==============================================================================\n# 目的：\n# 啟動一個網頁圖形化介面 (GUI)，讓您可以更直觀地設定參數並執行回測。\n#\n# 執行動作：\n# 1. 啟動伺服器：在背景執行 Gradio 網頁應用程式。\n# 2. 產生公開網址：Gradio 會自動產生一個可分享的公開網址 (URL)。\n#\n# 如何使用：\n# 1. 執行此儲存格。\n# 2. 在輸出區找到類似 `Running on public URL: https://....gradio.live` 的網址。\n# 3. **點擊該網址**，會在新的瀏覽器分頁中開啟儀表板介面。\n# 4. 在新開啟的網頁中設定參數並執行回測。\n# ==============================================================================\n\nimport os\n\nGITHUB_REPO_URL = os.environ.get('GITHUB_REPO_URL')\nif not GITHUB_REPO_URL:\n    raise ValueError("錯誤：找不到 GITHUB_REPO_URL。請先執行儲存格 2。")\n\nrepo_name = GITHUB_REPO_URL.split('/')[-1].replace('.git', '')\napp_path = f"/content/{repo_name}/lo2cin4bt/app.py"\n\nif not os.path.exists(app_path):\n    raise FileNotFoundError(f"錯誤：找不到應用程式腳本 '{app_path}'。")\n\n# 使用 -m 將 lo2cin4bt 作為模組執行，以確保相對匯入正確\n# 我們需要先進入專案資料夾，才能正確地以模組方式執行\n%cd /content/{repo_name}\n!python -m lo2cin4bt.app

In [ ]:
#@title 5. (可選) 同步成果至 Google Drive { display-mode: "form" }\n\nimport os\nfrom google.colab import drive\nfrom distutils.dir_util import copy_tree\n\ngdrive_path = os.environ.get('GDRIVE_PROJECT_PATH')\nproject_root = os.environ.get('PROJECT_ROOT_PATH')\n\nif not project_root or not os.path.exists(project_root):\n    print("❌ 錯誤：找不到任何本地專案成果。")\nelif gdrive_path and project_root == gdrive_path:\n    print("✅ 您的專案成果已經儲存在 Google Drive 中，無需同步。")\nelse:\n    print("🔄 開始將本地成果同步至 Google Drive...")\n    if not gdrive_path:\n        try:\n            print("正在嘗試掛載 Google Drive...")\n            drive.mount('/content/drive', force_remount=True)\n            project_name = "MyQuantProject"\n            gdrive_path = os.path.join('/content/drive/MyDrive', project_name)\n            os.environ['GDRIVE_PROJECT_PATH'] = gdrive_path\n            print(f"✅ Google Drive 掛載成功！")\n        except Exception as e:\n            print(f"❌ 掛載 Google Drive 失敗。錯誤：{e}")\n            gdrive_path = None\n    if gdrive_path:\n        print(f"正在將 '{project_root}' 複製到 '{gdrive_path}'...")\n        os.makedirs(gdrive_path, exist_ok=True)\n        copy_tree(project_root, gdrive_path)\n        print("✅ 同步完成！")